In [1]:
import torchvision 
import numpy as np
import torch
import torch.autograd as autograd
from torch.autograd import Variable, Function
from torch.nn.parameter import Parameter
import torch.nn as nn
import pylab as pl
import time
import math
import pickle
from IPython import display
import torch.nn.functional as F
from copy import deepcopy
import cifarResnet
from torchsummary import summary
from matplotlib import pyplot as plt
from scipy.misc import bytescale

torch.cuda.set_device(3) # Set gpu number here

In [2]:
class SimpleConv(nn.Module):
    def __init__(self):
        super(SimpleConv, self).__init__()
        self.features = nn.Sequential(
            nn.Linear(5,3),
            nn.ReLU(inplace=True),
        )

    def forward(self, x,y):
        x = self.features(x)
        x2 = self.features(y)
        #x = torch.cat((x1, x2), 1)
        return x

In [5]:
class Conv(nn.Module):
    def __init__(self):
        super(Conv, self).__init__()
        layers = []
        for i in range(3):
            layers.append(SimpleConv())

        self.features = nn.Sequential(*layers)

    def forward(self, x,y,z):
        x = self.features(x,y)
        x2 = self.features(y,z)
        #x = torch.cat((x1, x2), 1)
        return x

In [9]:
model = Conv()
x = torch.ones(5)
out = model(x,x)
print(out)
print(model)

TypeError: forward() takes 2 positional arguments but 3 were given

In [2]:
model = cifarResnet.ResNet(9,16).cuda()
i = model.fc.in_features
model.fc = nn.Linear(i,100).cuda()

checkpoint = torch.load('/data/xay2/scripts/pretrained_resnet/fine_tune.pth')
model.load_state_dict(checkpoint)

# freeze the feature extractors 
for param in model.parameters():
    print(param, param.requires_grad)

this is block None
this is block 1
this is block 2
this is block 3
this is block 4
this is block 5
this is block 6
this is block 7
this is block 8
this is block Sequential(
  (0): Conv2d(16, 32, kernel_size=(1, 1), stride=(2, 2), bias=False)
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)
this is block 1
this is block 2
this is block 3
this is block 4
this is block 5
this is block 6
this is block 7
this is block 8
this is block Sequential(
  (0): Conv2d(32, 64, kernel_size=(1, 1), stride=(2, 2), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)
this is block 1
this is block 2
this is block 3
this is block 4
this is block 5
this is block 6
this is block 7
this is block 8


RuntimeError: Error(s) in loading state_dict for ResNet:
	Missing key(s) in state_dict: "layer2.0.index.0.weight", "layer2.0.index.1.weight", "layer2.0.index.1.bias", "layer2.0.index.1.running_mean", "layer2.0.index.1.running_var", "layer3.0.index.0.weight", "layer3.0.index.1.weight", "layer3.0.index.1.bias", "layer3.0.index.1.running_mean", "layer3.0.index.1.running_var". 
	Unexpected key(s) in state_dict: "layer2.0.downsample.0.weight", "layer2.0.downsample.1.weight", "layer2.0.downsample.1.bias", "layer2.0.downsample.1.running_mean", "layer2.0.downsample.1.running_var", "layer3.0.downsample.0.weight", "layer3.0.downsample.1.weight", "layer3.0.downsample.1.bias", "layer3.0.downsample.1.running_mean", "layer3.0.downsample.1.running_var". 